In [ ]:
# =============================================================================
# 🚀 그룹 1: 통일된 함수명 - 마이리얼트립 크롤링 시스템 (함수명 정리 완료)
# 함수명 단순화: get_product_name(), get_price(), download_image(), clean_price(), clean_rating(), save_results()
# =============================================================================

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')

import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용  
from datetime import datetime    # 타임스탬프용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 5,                  # 3 → 5초로 증가
    "MAX_DELAY": 12,                 # 8 → 12초로 증가
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30,
    "LONGER_DELAYS": True,           # 새로 추가
    "MEMORY_CLEANUP_INTERVAL": 5,    # 새로 추가
    "MAX_PRODUCTS_PER_CITY": 2,     # 2 → 10개로 증가⭐⭐⭐⭐⭐⭐⭐⭐⭐
    # 🆕 Gemini 지적사항 해결: USER_AGENT 추가
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["후쿠오카"]

# 🆕 통합된 도시 정보 구조 (Gemini 지적사항 반영)
CITY_CODES = {
    # 동남아시아
    "방콕": "BKK",
    "치앙마이": "CNX", 
    "푸켓": "HKT",
    "싱가포르": "SIN",
    "홍콩": "HKG",
    "쿠알라룸푸르": "KUL",
    "세부": "CEB",
    "다낭": "DAD",
    "호치민": "SGN",
    
    # 일본
    "도쿄": "NRT",
    "오사카": "KIX",
    "나고야": "NGO",
    "후쿠오카": "FUK",
    "오키나와": "OKA",
    "삿포로": "CTS",
    
    # 한국
    "서울": "ICN",
    "부산": "PUS",
    "제주": "CJU",
    "대구": "TAE",
    "광주": "KWJ",
    "여수": "RSU",
    
    # 유럽
    "파리": "CDG",
    "런던": "LHR",
    "로마": "FCO",
    "바르셀로나": "BCN",
    
    # 북미
    "뉴욕": "JFK",
    "로스앤젤레스": "LAX",
    "시카고": "ORD",
    
    # 오세아니아
    "시드니": "SYD",
    "멜버른": "MEL",
}

# 🆕 통합된 도시 정보 (대륙/국가 정보 포함)
UNIFIED_CITY_INFO = {
    "방콕": {"대륙": "아시아", "국가": "태국", "코드": "BKK"},
    "도쿄": {"대륙": "아시아", "국가": "일본", "코드": "NRT"},
    "오사카": {"대륙": "아시아", "국가": "일본", "코드": "KIX"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르", "코드": "SIN"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩", "코드": "HKG"},
    "파리": {"대륙": "유럽", "국가": "프랑스", "코드": "CDG"},
    "런던": {"대륙": "유럽", "국가": "영국", "코드": "LHR"},
    "뉴욕": {"대륙": "북미", "국가": "미국", "코드": "JFK"},
    "시드니": {"대륙": "오세아니아", "국가": "호주", "코드": "SYD"},
}

print(f"✅ CITY_CODES 추가 완료! {len(CITY_CODES)}개 도시 지원")

# =============================================================================
# 🔧 핵심 함수들 - 함수명 통일 완료! (마이리얼트립 전용)
# =============================================================================

def get_city_code(city_name):
    """도시명으로 공항 코드 반환 (공용 함수)"""
    code = CITY_CODES.get(city_name, city_name[:3].upper())
    print(f"  🏙️ {city_name} → {code}")
    return code

def get_city_info(city_name):
    """통합된 도시 정보 가져오기 (공용 함수)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        return info["대륙"], info["국가"]
    else:
        # 기본값 반환
        return "기타", "기타"

def get_product_name(driver, url_type="Product"):
    """✅ 상품명 수집 (기존: get_product_name_by_type → 새로운: get_product_name)"""
    print(f"  📊 {url_type} 상품명 수집 중...")
    
    # 기본 상품명 셀렉터들 (Products와 Offers 공통)
    title_selectors = [
        (By.CSS_SELECTOR, "h1"),
        (By.CSS_SELECTOR, ".product-title"),
        (By.XPATH, "//h1[contains(@class, 'title')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver):
    """✅ 가격 수집 (기존: get_price_fixed → 새로운: get_price)"""
    print(f"  💰 가격 정보 수집 중...")
    
    # 마이리얼트립 실제 가격 위치들 (우선순위 순서)
    price_selectors = [
        # 1순위: 할인된 가격 (빨간색 텍스트)
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 87, 87)']"),
        (By.CSS_SELECTOR, "span[style*='color: red']"),
        (By.CSS_SELECTOR, ".price-discount"),
        
        # 2순위: 일반 가격
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        
        # 3순위: 원이 포함된 텍스트 (쿠폰 제외)
        (By.XPATH, "//span[contains(text(), '원') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        
        # 4순위: 기본 가격 위치
        (By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div/div[2]/span[2]")
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_price = price_element.text.strip()
            
            # 쿠폰 관련 텍스트 제외
            if any(keyword in found_price for keyword in ['쿠폰', '받기', '다운']):
                continue
                
            # 가격 패턴 확인 (숫자 + 원)
            if '원' in found_price and any(char.isdigit() for char in found_price):
                return found_price
                
        except TimeoutException:
            continue
    
    return "정보 없음"

def clean_price(price_text):
    """✅ 가격 정제 (기존: extract_clean_price → 새로운: clean_price) (공용 함수)"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    # 가격 패턴: 숫자,숫자원 또는 숫자원
    price_pattern = r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?'
    match = re.search(price_pattern, price_text)
    
    if match:
        return match.group(1) + "원"
    else:
        return price_text  # 원본 반환

def clean_rating(rating_text):
    """✅ 평점 정제 (기존: extract_clean_rating → 새로운: clean_rating) (공용 함수)"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    # 평점 패턴: 숫자.숫자
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return rating_text
    else:
        return rating_text

def download_image(driver, product_name, city_name, product_index):
    """✅ 이미지 다운로드 (기존: download_image_improved_fixed → 새로운: download_image)"""
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': '',
            'path': '',
            'size': 0
        }
        
    print(f"  🖼️ 대표 상품 이미지 다운로드 중...")
    
    # 대표 이미지 우선 셀렉터들
    image_selectors = [
        # 1순위: 큰 메인 이미지
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        
        # 2순위: 일반 상품 이미지
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        
        # 3순위: 기본 이미지
        (By.XPATH, "//img[contains(@alt, '상품')]"),
        (By.CSS_SELECTOR, "img[src*='cdn']"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            
            for img_element in img_elements:
                img_url = img_element.get_attribute('src')
                if img_url and img_url.startswith('http'):
                    # 이미지 크기 확인 (너무 작은 이미지 제외)
                    try:
                        width = img_element.get_attribute('width') or img_element.get_attribute('naturalWidth')
                        height = img_element.get_attribute('height') or img_element.get_attribute('naturalHeight')
                        
                        if width and height:
                            if int(width) < 100 or int(height) < 100:
                                continue  # 너무 작은 이미지는 스킵
                    except:
                        pass
                    
                    break
            
            if img_url:
                break
                
        except Exception:
            continue

    if img_url:
        try:
            # 공항 코드 기반 파일명
            city_code = get_city_code(city_name)
            img_filename = f"{city_code}_{product_index:03d}.jpg"
            
            # 이미지 폴더 생성
            img_folder = "myrealtripthumb_img"
            os.makedirs(img_folder, exist_ok=True)
            
            img_path = os.path.join(img_folder, img_filename)
            
            # 이미지 다운로드
            urllib.request.urlretrieve(img_url, img_path)
            
            # 파일 크기 확인
            file_size = os.path.getsize(img_path)
            
            if file_size < 1024:  # 1KB 미만이면 실패로 간주
                os.remove(img_path)
                return {
                    'status': '다운로드 실패 (파일 너무 작음)',
                    'filename': img_filename,
                    'path': '',
                    'size': 0
                }
            
            print(f"  ✅ 이미지 다운로드 완료! ({file_size:,} bytes)")
            return {
                'status': '다운로드 완료',
                'filename': img_filename,
                'path': img_path,
                'size': file_size
            }
            
        except Exception as e:
            print(f"  ⚠️ 이미지 다운로드 실패: {type(e).__name__}")
            return {
                'status': f'다운로드 실패: {type(e).__name__}',
                'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
                'path': '',
                'size': 0
            }
    else:
        return {
            'status': '이미지 없음',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'size': 0
        }

def save_results(products_data):
    """✅ 데이터 저장 (기존: save_myrealtrip_data → 새로운: save_results)"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 도시명 가져오기
    city_name = products_data[0]['도시'] if products_data else 'unknown'
    
    # 개별 CSV 저장
    df = pd.DataFrame(products_data)
    csv_path = f"myrealtrip_{city_name}_products_{len(products_data)}개_{timestamp}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"📁 개별 CSV 저장 완료: {csv_path}")
    
    # 메타데이터 저장
    metadata = {
        "myrealtrip": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "city": city_name
        }
    }
    
    try:
        with open('data_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        print(f"📁 메타데이터 저장 완료: data_metadata.json")
    except Exception as e:
        print(f"⚠️ 메타데이터 저장 실패: {e}")
    
    return csv_path

# =============================================================================
# 🚀 Phase 2: 확장성 개선 시스템 (기존 코드 유지)
# =============================================================================

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장"""
    
    # 🆕 Gemini 지적사항 반영: 대륙/국가 정보도 포함
    enhanced_city_data = {
        "version": "2.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": {},
        "total_cities": len(CITY_CODES)
    }
    
    # 통합된 정보로 cities 구성
    for city_name, city_code in CITY_CODES.items():
        city_info = UNIFIED_CITY_INFO.get(city_name, {"대륙": "기타", "국가": "기타"})
        enhanced_city_data["cities"][city_name] = {
            "code": city_code,
            "continent": city_info["대륙"],
            "country": city_info["국가"]
        }
    
    try:
        with open('city_codes.json', 'w', encoding='utf-8') as f:
            json.dump(enhanced_city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ city_codes.json 파일 생성 완료! ({len(CITY_CODES)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드"""
    
    if not os.path.exists('city_codes.json'):
        print("📝 city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
        return CITY_CODES
    
    try:
        with open('city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        # 새 형식 (v2.0) 처리
        if "cities" in city_data and isinstance(list(city_data["cities"].values())[0], dict):
            loaded_codes = {city: info["code"] for city, info in city_data["cities"].items()}
        else:
            # 구 형식 (v1.0) 처리
            loaded_codes = city_data.get('cities', {})
        
        print(f"✅ city_codes.json 로드 완료! ({len(loaded_codes)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
        return loaded_codes
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 기존 코드의 CITY_CODES를 사용합니다.")
        return CITY_CODES

def add_new_city(city_name, airport_code, update_file=True):
    """새로운 도시를 추가하는 함수"""
    
    global CITY_CODES
    
    # 메모리에 추가
    CITY_CODES[city_name] = airport_code
    print(f"✅ 메모리에 추가: {city_name} → {airport_code}")
    
    # 파일에도 저장
    if update_file:
        create_city_codes_file()  # 전체 재생성
    
    return True

def show_supported_cities():
    """지원하는 도시 목록 표시"""
    
    print("\\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    # 지역별로 분류
    regions = {
        "동남아시아": ["방콕", "치앙마이", "푸켓", "싱가포르", "홍콩", "쿠알라룸푸르", "세부", "다낭", "호치민"],
        "일본": ["도쿄", "오사카", "나고야", "후쿠오카", "오키나와", "삿포로"],
        "한국": ["서울", "부산", "제주", "대구", "광주", "여수"],
        "유럽": ["파리", "런던", "로마", "바르셀로나"],
        "북미": ["뉴욕", "로스앤젤레스", "시카고"],
        "오세아니아": ["시드니", "멜버른"]
    }
    
    for region, cities in regions.items():
        print(f"\\n📍 {region}:")
        for city in cities:
            if city in CITY_CODES:
                code = CITY_CODES[city]
                print(f"   {city} → {code}")
    
    print(f"\\n📊 총 {len(CITY_CODES)}개 도시 지원")
    print("="*50)

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    
    global CONFIG
    
    # 기존 CONFIG에 확장성 설정 추가
    scalability_config = {
        # 도시 관리
        "AUTO_LOAD_CITIES": True,
        "AUTO_SAVE_NEW_CITIES": True,
        
        # 다중 도시 지원
        "ENABLE_MULTI_CITY": False,
        "CITY_PROCESSING_ORDER": "sequential",
        
        # 파일 관리
        "BACKUP_OLD_DATA": True,
        "DATA_RETENTION_DAYS": 30,
        
        # 확장 기능
        "ENABLE_CITY_VALIDATION": True,
        "ENABLE_DUPLICATE_CHECK": True,
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

def initialize_file_system():
    """파일 시스템 초기화 및 설정"""
    
    print("🔧 Phase 2: 확장성 개선 시스템 초기화...")
    
    # CONFIG 업데이트
    update_config_for_scalability()
    
    # 도시 코드 파일 로드/생성
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        global CITY_CODES
        loaded_codes = load_city_codes_from_file()
        
        # 새로 로드된 코드가 더 많으면 업데이트
        if len(loaded_codes) >= len(CITY_CODES):
            CITY_CODES = loaded_codes
            print(f"🔄 CITY_CODES 업데이트: {len(CITY_CODES)}개 도시")
        else:
            # 메모리의 코드가 더 최신이면 파일 업데이트
            create_city_codes_file()
    
    print("✅ Phase 2 시스템 초기화 완료!")
    return True

def quick_add_cities():
    """자주 사용하는 도시들을 빠르게 추가"""
    
    quick_cities = {
        # 추가로 자주 사용될 도시들
        "교토": "KIX",      # 오사카 공항 사용
        "인천": "ICN",      # 서울 공항
        "김포": "GMP",      # 김포공항
        "하와이": "HNL",    # 호놀룰루
        "괌": "GUM",        # 괌 국제공항
        "사이판": "SPN",    # 사이판 공항
        "푸꾸옥": "PQC",    # 푸꾸옥 공항
        "나트랑": "CXR",    # 나트랑 공항
        "보홀": "TAG",      # 보홀 공항
        "랑카위": "LGK",    # 랑카위 공항
    }
    
    print("🚀 자주 사용하는 도시들 추가 중...")
    
    for city, code in quick_cities.items():
        if city not in CITY_CODES:
            add_new_city(city, code, update_file=False)
    
    # 한 번에 파일 저장
    create_city_codes_file()
    
    print(f"✅ {len(quick_cities)}개 도시 일괄 추가 완료!")

def validate_city(city_name):
    """도시명 유효성 검사"""
    
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in CITY_CODES:
        return True, f"지원하는 도시입니다. ({CITY_CODES[city_name]})"
    
    # 유사한 도시명 찾기
    similar_cities = []
    for supported_city in CITY_CODES.keys():
        if city_name.lower() in supported_city.lower() or supported_city.lower() in city_name.lower():
            similar_cities.append(supported_city)
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. 새로 추가하시려면 add_new_city() 함수를 사용하세요."

# =============================================================================
# 🛠️ 기존 유틸리티 함수들 (Phase 2 시스템과 함께 유지)
# =============================================================================

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG["SAVE_INTERMEDIATE"]:
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]  # ✅ Gemini 지적사항 해결: USER_AGENT 사용
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        shutil.rmtree(raw_path)
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
    except Exception as e:
        print('\n',"-"*50,"\n","-"*50,"\n")
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,"\n","-"*50,"\n")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            view_all_button.click()
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
    return True

def collect_page_urls(driver):
    """현재 페이지의 모든 상품 URL 수집"""
    print(f"  📊 현재 페이지의 상품 URL들을 수집 중...")
    
    time.sleep(random.uniform(3, 5))
    
    product_url_selectors = [
        "a[href*='/experiences/']",
        "a[href*='/experience/']",
        ".product-item a",
        ".experience-card a"
    ]
    
    collected_urls = []
    
    for selector in product_url_selectors:
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in product_elements:
                try:
                    url = element.get_attribute('href')
                    if url and '/experiences/' in url and url not in collected_urls:
                        collected_urls.append(url)
                except Exception as e:
                    continue
            
            if collected_urls:
                break
                
        except Exception as e:
            continue
    
    valid_urls = []
    for url in collected_urls:
        if url and url.startswith('http') and '/experiences/' in url:
            valid_urls.append(url)
    
    print(f"  ✅ {len(valid_urls)}개의 상품 URL을 수집했습니다!")
    
    if len(valid_urls) == 0:
        print("  ⚠️ 상품 URL을 찾을 수 없습니다. 페이지 구조를 확인해주세요.")
    
    return valid_urls

def get_rating(driver):
    """평점 정보 수집 (기존 함수 유지)"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집"""
    print(f"  🌐 언어 정보 찾는 중...")
    language_selectors = [
        (By.XPATH, "//span[contains(text(), '언어')]"),
        (By.XPATH, "//span[contains(text(), '한국어')]"),
        (By.XPATH, "//span[contains(text(), '영어')]"),
        (By.XPATH, "//span[contains(text(), 'Korean')]"),
        (By.XPATH, "//span[contains(text(), 'English')]"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            language_text = language_element.text.strip()
            
            language_keywords = ['언어', '한국어', '영어', '중국어', '일본어', 'Korean', 'English']
            
            if any(keyword in language_text for keyword in language_keywords):
                print(f"  ✅ 언어 정보 발견: {language_text}")
                return language_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

# =============================================================================
# Phase 2 시스템 자동 실행
# =============================================================================

# 자동 초기화 실행
try:
    initialize_file_system()
    
    # 자주 사용하는 도시들도 추가
    quick_add_cities()
    
    # 지원 도시 목록 표시
    show_supported_cities()
    
    print("\n🎉 Phase 2: 확장성 개선 완료!")
    print("💡 이제 이런 기능들을 사용할 수 있습니다:")
    print("   - add_new_city('제주도', 'CJU')  # 새 도시 추가")
    print("   - show_supported_cities()        # 지원 도시 목록")
    print("   - validate_city('방콕')          # 도시 유효성 검사")
    
except Exception as e:
    print(f"❌ Phase 2 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")

print("\n" + "="*60)
print("✅ 그룹 1 완료: 모든 함수가 정의되었습니다!")
print("="*60)
print("🔧 핵심 함수명 변경 완료:")
print("   get_product_name_by_type() → get_product_name()")
print("   get_price_fixed() → get_price()")
print("   download_image_improved_fixed() → download_image()")
print("   extract_clean_price() → clean_price()")
print("   extract_clean_rating() → clean_rating()")
print("   save_myrealtrip_data() → save_results()")
print("="*60)
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print("🎯 다음: 그룹 2,3,4에서 함수 호출을 통일된 함수명으로 변경하세요!")
print("🚨 Gemini 지적사항 모두 해결 완료! 안전하게 실행 가능합니다!")

In [ ]:
# =============================================================================
# 🔍 그룹 2: 통일된 함수명 - 검색 및 URL 수집 시스템 (함수명 통일 완료)
# 그룹 1에서 정의된 통일된 함수명들을 사용
# =============================================================================

print("🔍 그룹 2: 검색 및 URL 수집 시스템 시작!")
print("📋 그룹 1의 통일된 함수명들을 사용합니다:")
print("   - get_product_name()")
print("   - get_price()")
print("   - download_image()")
print("   - clean_price()")
print("   - clean_rating()")
print("   - save_results()")
print("   - get_city_code(), get_city_info() (공용)")

def search_and_collect_urls(driver, city_name):
    """도시별 검색 및 상품 URL 수집 (통일된 함수명 사용)"""
    print(f"\n🔍 {city_name} 도시 검색 시작...")
    
    try:
        # 1. 메인 페이지 이동
        print("📍 1단계: 메인 페이지 이동")
        go_to_main_page(driver)
        
        # 2. 검색창 찾기 및 입력
        print("📍 2단계: 검색 실행")
        find_and_fill_search(driver, city_name)
        
        # 3. 검색 버튼 클릭
        click_search_button(driver)
        
        # 4. 팝업 처리
        print("📍 3단계: 팝업 처리")
        handle_popup(driver)
        
        # 5. 전체 상품 보기 클릭
        print("📍 4단계: 전체 상품 보기")
        click_view_all(driver)
        
        # 6. 상품 URL 수집
        print("📍 5단계: 상품 URL 수집")
        product_urls = collect_page_urls(driver)
        
        # 7. 수집 결과 요약
        print(f"\n✅ {city_name} 검색 완료!")
        print(f"📊 수집된 상품 URL: {len(product_urls)}개")
        
        # 8. 도시 코드 확인 (통일된 함수명 사용)
        city_code = get_city_code(city_name)
        continent, country = get_city_info(city_name)
        
        print(f"🏙️ 도시 정보: {city_name} ({city_code}) - {continent}, {country}")
        
        return product_urls
        
    except Exception as e:
        print(f"❌ {city_name} 검색 실패: {type(e).__name__}: {e}")
        return []

def validate_and_filter_urls(product_urls, city_name):
    """수집된 URL 유효성 검사 및 필터링"""
    print(f"\n🔍 {city_name} URL 유효성 검사 중...")
    
    valid_urls = []
    invalid_urls = []
    
    for i, url in enumerate(product_urls):
        print(f"  📋 URL {i+1}/{len(product_urls)} 검사 중...")
        
        # URL 기본 유효성 검사
        if not url or not url.startswith('http'):
            invalid_urls.append(url)
            continue
            
        # 마이리얼트립 경험 상품 URL 확인
        if '/experiences/' not in url:
            invalid_urls.append(url)
            continue
            
        # 중복 제거
        if url in valid_urls:
            continue
            
        valid_urls.append(url)
    
    print(f"  ✅ 유효한 URL: {len(valid_urls)}개")
    print(f"  ❌ 무효한 URL: {len(invalid_urls)}개")
    
    return valid_urls

def prepare_crawling_data(valid_urls, city_name):
    """크롤링을 위한 데이터 준비"""
    print(f"\n🔧 {city_name} 크롤링 데이터 준비 중...")
    
    # 최대 상품 수 제한
    max_products = CONFIG["MAX_PRODUCTS_PER_CITY"]
    if len(valid_urls) > max_products:
        valid_urls = valid_urls[:max_products]
        print(f"  ⚠️ 상품 수를 {max_products}개로 제한했습니다.")
    
    # 크롤링 준비 데이터 생성
    crawling_data = []
    
    for i, url in enumerate(valid_urls):
        # 도시 코드 생성 (통일된 함수명 사용)
        city_code = get_city_code(city_name)
        continent, country = get_city_info(city_name)
        
        item_data = {
            'index': i + 1,
            'url': url,
            'city_name': city_name,
            'city_code': city_code,
            'continent': continent,
            'country': country,
            'status': 'pending'
        }
        
        crawling_data.append(item_data)
    
    print(f"  ✅ {len(crawling_data)}개 상품 크롤링 데이터 준비 완료!")
    
    return crawling_data

def quick_url_validation(driver, sample_url):
    """샘플 URL로 빠른 유효성 검사"""
    print(f"\n🔍 샘플 URL 유효성 검사 중...")
    
    try:
        # 샘플 URL 접속
        driver.get(sample_url)
        time.sleep(random.uniform(2, 4))
        
        # 기본 요소들 확인 (통일된 함수명 사용)
        try:
            # 상품명 존재 확인
            test_name = get_product_name(driver)
            print(f"  ✅ 상품명 수집 가능: {test_name[:30]}...")
            
            # 가격 정보 확인
            test_price = get_price(driver)
            print(f"  ✅ 가격 정보 수집 가능: {test_price}")
            
            # 가격 정제 테스트 (통일된 함수명 사용)
            clean_price_result = clean_price(test_price)
            print(f"  ✅ 가격 정제 결과: {clean_price_result}")
            
        except Exception as e:
            print(f"  ⚠️ 상품 정보 수집 실패: {type(e).__name__}")
            return False
        
        print(f"  ✅ URL 유효성 검사 통과!")
        return True
        
    except Exception as e:
        print(f"  ❌ URL 접속 실패: {type(e).__name__}: {e}")
        return False

def multi_city_search(driver, cities_list):
    """다중 도시 검색 (통일된 함수명 사용)"""
    print(f"\n🌍 다중 도시 검색 시작! ({len(cities_list)}개 도시)")
    
    all_crawling_data = []
    
    for i, city_name in enumerate(cities_list):
        print(f"\n{'='*60}")
        print(f"🔍 도시 {i+1}/{len(cities_list)}: {city_name}")
        print(f"{'='*60}")
        
        # 도시 유효성 검사 (통일된 함수명 사용)
        is_valid, message = validate_city(city_name)
        if not is_valid:
            print(f"  ⚠️ {message}")
            continue
        
        try:
            # 1. 검색 및 URL 수집
            product_urls = search_and_collect_urls(driver, city_name)
            
            if not product_urls:
                print(f"  ⚠️ {city_name}에서 상품 URL을 찾을 수 없습니다.")
                continue
            
            # 2. URL 유효성 검사
            valid_urls = validate_and_filter_urls(product_urls, city_name)
            
            if not valid_urls:
                print(f"  ⚠️ {city_name}에서 유효한 URL을 찾을 수 없습니다.")
                continue
            
            # 3. 크롤링 데이터 준비
            city_crawling_data = prepare_crawling_data(valid_urls, city_name)
            
            # 4. 샘플 URL 유효성 검사
            if city_crawling_data:
                sample_url = city_crawling_data[0]['url']
                if quick_url_validation(driver, sample_url):
                    all_crawling_data.extend(city_crawling_data)
                    print(f"  ✅ {city_name} 검색 완료! {len(city_crawling_data)}개 상품 준비")
                else:
                    print(f"  ❌ {city_name} URL 유효성 검사 실패")
            
            # 5. 중간 결과 저장 (통일된 함수명 사용)
            if CONFIG.get("SAVE_INTERMEDIATE", False):
                temp_data = [{'도시': city_name, 'URL수': len(valid_urls), '준비완료': len(city_crawling_data)}]
                save_intermediate_results(temp_data, f"{city_name}_search_results")
            
        except Exception as e:
            print(f"  ❌ {city_name} 검색 중 오류 발생: {type(e).__name__}: {e}")
            continue
        
        # 도시 간 대기 시간
        if i < len(cities_list) - 1:
            delay = random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"])
            print(f"  💤 다음 도시 검색 전 대기: {delay:.1f}초")
            time.sleep(delay)
    
    # 최종 결과 요약
    print(f"\n{'='*60}")
    print(f"🎉 다중 도시 검색 완료!")
    print(f"{'='*60}")
    print(f"📊 검색 도시: {len(cities_list)}개")
    print(f"📊 총 수집 상품: {len(all_crawling_data)}개")
    
    # 도시별 상품 수 요약
    city_summary = {}
    for item in all_crawling_data:
        city = item['city_name']
        city_summary[city] = city_summary.get(city, 0) + 1
    
    print(f"\n📋 도시별 상품 수:")
    for city, count in city_summary.items():
        city_code = get_city_code(city)  # 통일된 함수명 사용
        print(f"   {city} ({city_code}): {count}개")
    
    return all_crawling_data

def save_search_results(all_crawling_data):
    """검색 결과를 임시 저장 (통일된 함수명 사용)"""
    if not all_crawling_data:
        print("  ⚠️ 저장할 검색 결과가 없습니다.")
        return None
    
    print(f"\n💾 검색 결과 임시 저장 중...")
    
    # 검색 결과 데이터 변환
    search_results = []
    for item in all_crawling_data:
        result = {
            '순번': item['index'],
            '도시': item['city_name'],
            '도시코드': item['city_code'],
            '대륙': item['continent'],
            '국가': item['country'],
            'URL': item['url'],
            '상태': item['status']
        }
        search_results.append(result)
    
    # 임시 저장 (통일된 함수명의 save_intermediate_results 사용)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    temp_filename = f"search_results_{len(search_results)}개_{timestamp}.csv"
    
    try:
        df = pd.DataFrame(search_results)
        df.to_csv(temp_filename, index=False, encoding='utf-8-sig')
        print(f"  ✅ 검색 결과 저장 완료: {temp_filename}")
        return temp_filename
    except Exception as e:
        print(f"  ⚠️ 검색 결과 저장 실패: {e}")
        return None

def run_group2_search():
    """그룹 2 검색 시스템 실행"""
    print(f"\n🚀 그룹 2: 검색 시스템 실행 시작!")
    print(f"📋 검색 대상 도시: {CITIES_TO_SEARCH}")
    
    # 드라이버 설정
    driver = None
    try:
        # 1. 드라이버 초기화
        driver = setup_driver()
        
        # 2. 다중 도시 검색 실행
        all_crawling_data = multi_city_search(driver, CITIES_TO_SEARCH)
        
        if not all_crawling_data:
            print("❌ 검색 결과가 없습니다.")
            return None
        
        # 3. 검색 결과 저장
        saved_file = save_search_results(all_crawling_data)
        
        # 4. 최종 결과 출력
        print(f"\n🎉 그룹 2 검색 완료!")
        print(f"📊 총 {len(all_crawling_data)}개 상품 URL 수집")
        print(f"📁 저장된 파일: {saved_file}")
        print(f"🎯 다음: 그룹 3 (URL 수집) 또는 그룹 4 (메인 크롤링) 실행")
        
        return all_crawling_data
        
    except Exception as e:
        print(f"❌ 그룹 2 검색 실행 중 오류: {type(e).__name__}: {e}")
        return None
        
    finally:
        # 드라이버 종료
        if driver:
            try:
                driver.quit()
                print("✅ 드라이버 종료 완료")
            except:
                pass

def test_unified_functions():
    """통일된 함수명 테스트"""
    print(f"\n🔧 통일된 함수명 테스트 시작...")
    
    # 도시 코드 테스트
    test_cities = ["후쿠오카", "도쿄", "방콕"]
    
    for city in test_cities:
        print(f"\n📍 {city} 테스트:")
        
        # 통일된 함수명 사용
        city_code = get_city_code(city)
        continent, country = get_city_info(city)
        
        print(f"  🏙️ 도시 코드: {city_code}")
        print(f"  🌍 위치: {continent}, {country}")
        
        # 도시 유효성 검사
        is_valid, message = validate_city(city)
        print(f"  ✅ 유효성: {is_valid} - {message}")
    
    # 가격 정제 테스트
    test_prices = [
        "15,000원",
        "최대 5만원 할인 + 25,000원",
        "3,500원부터",
        "정보 없음"
    ]
    
    print(f"\n💰 가격 정제 테스트:")
    for price in test_prices:
        # 통일된 함수명 사용
        clean_result = clean_price(price)
        print(f"  원본: {price} → 정제: {clean_result}")
    
    # 평점 정제 테스트
    test_ratings = [
        "4.9 · 후기 많음",
        "4.2",
        "평점 없음",
        "5.0 ★★★★★"
    ]
    
    print(f"\n⭐ 평점 정제 테스트:")
    for rating in test_ratings:
        # 통일된 함수명 사용
        clean_result = clean_rating(rating)
        print(f"  원본: {rating} → 정제: {clean_result}")
    
    print(f"\n✅ 통일된 함수명 테스트 완료!")

# =============================================================================
# 그룹 2 실행 옵션
# =============================================================================

print("\n" + "="*60)
print("✅ 그룹 2 완료: 검색 시스템 준비 완료!")
print("="*60)
print("🔧 사용 가능한 함수들:")
print("   - search_and_collect_urls(driver, city_name)")
print("   - multi_city_search(driver, cities_list)")
print("   - run_group2_search()  # 전체 검색 실행")
print("   - test_unified_functions()  # 함수명 테스트")
print("="*60)
print("🎯 실행 방법:")
print("   1. test_unified_functions()      # 함수 테스트")
print("   2. run_group2_search()           # 전체 검색 실행")
print("   3. 또는 그룹 4에서 직접 크롤링 실행")
print("="*60)
print("✅ 모든 함수가 그룹 1의 통일된 함수명을 사용합니다!")
print("   get_product_name(), get_price(), clean_price(), clean_rating(), save_results() 등")